In [ ]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 95% !important}</style>'))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("classic")
import yfinance as yf
from datetime import date
import pandas_ta as ta
from dateutil.relativedelta import relativedelta

In [ ]:
end = date.today() # todays date
start = end + relativedelta(months=-6) # 6 months past period
data = yf.download(tickers="ETH-USD", start=start, end=end, interval="1h") # you can set up relevant interval for future optimisation

In [ ]:
# preparing initial dataset
data = data.drop(columns=["Open", "Adj Close", "Volume"])
data["position"] = 0

In [ ]:
# implementation of indicator EMA
data["ema_fast"] = ta.ema(close=data["Close"], length=5)
data["ema_slow"] = ta.ema(close=data["Close"], length=24)
data["ema_cross"] = data["ema_fast"] - data["ema_slow"]

In [ ]:
# implementation of trade position with price to dataset
data["position"] = np.where(data["ema_cross"] > 0, "long", "short") 

data["trade"] = np.where(((data["position"] == "long") & (data["position"].shift(periods=1) == "short")),
                         "long", np.where(((data["position"] == "short") & (data["position"].shift(periods=1) == "long")),
                         "short", "Nan"))

data["position_price"] = np.where((data["trade"] == "long") | (data["trade"] == "short"),
                                 data["Close"], 0)

In [ ]:
print(data.to_string())

In [ ]:
# checking average candle deviation for TP position

candle = data[["High", "Low", "Close"]]
candle["long_position"] = (candle["High"] / candle["Close"] - 1)
candle["short_position"] = (candle["Low"] / candle["Close"] - 1)
long_tp = candle.long_position.mean()
short_tp = candle.short_position.mean()

In [ ]:
# setting up TP value
data["tp_order"] = np.where(data["trade"] == "long", data["position_price"] * (1 + long_tp),
                           np.where(data["trade"] == "short", data["position_price"] * (1 + short_tp), "Nan"))

In [ ]:
print(data.to_string())